![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/jupyter/training/italian/Train-Lemmatizer-Italian.ipynb)

## 0. Colab Setup

In [1]:
# This is only to setup PySpark and Spark NLP on Colab
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

openjdk version "1.8.0_252"
OpenJDK Runtime Environment (build 1.8.0_252-8u252-b09-1~18.04-b09)
OpenJDK 64-Bit Server VM (build 25.252-b09, mixed mode)
     |████████████████████████████████| 215.7MB 53kB/s 
     |████████████████████████████████| 204kB 34.7MB/s 
     |████████████████████████████████| 122kB 2.8MB/s 


# Training Lemmatizer Model in Italian language

### A brief explaination about `Lemmatizer` annotator in Spark NLP:

Retrieves lemmas out of words with the objective of returning a base dictionary word<br><br>
**Type:** Token<br>
**Requires:** None<br>
**Input:** abduct -> abducted abducting abduct abducts<br><br>
**Functions:**<br>
* setDictionary(path, keyDelimiter, valueDelimiter, readAs, options): Path and options to lemma dictionary, in lemma vs possible words format. readAs can be LINE_BY_LINE or SPARK_DATASET. options contain option passed to spark reader if readAs is SPARK_DATASET.
<br>

**Example:**
```Python
lemmatizer = Lemmatizer() \
  .setInputCols(["token"]) \
  .setOutputCol("lemma") \
  .setDictionary("./lemmas001.txt")
```

Let's import required libraries including `SQL` and `ML` from Spark and some annotators from Spark NLP

In [ ]:
#Spark ML and SQL
from pyspark.ml import Pipeline, PipelineModel
from pyspark.sql.functions import array_contains
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
#Spark NLP
import sparknlp
from sparknlp.annotator import *
from sparknlp.common import RegexRule
from sparknlp.base import DocumentAssembler, Finisher

### Let's create a Spark Session for our app

In [3]:
spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)


Spark NLP version:  2.5.0
Apache Spark version:  2.4.4


In [4]:
! wget -N https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/it/lemma/dxc.technology/lemma_italian.txt -P /tmp

--2020-05-16 14:52:59--  https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/it/lemma/dxc.technology/lemma_italian.txt
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.131.53
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.131.53|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6900964 (6.6M) [text/plain]
Saving to: ‘/tmp/lemma_italian.txt’


lemma_italian.txt     0%[                    ]       0  --.-KB/s               
lemma_italian.txt   100%[===================>]   6.58M  --.-KB/s    in 0.1s    

2020-05-16 14:52:59 (54.5 MB/s) - ‘/tmp/lemma_italian.txt’ saved [6900964/6900964]



### Now we are going to create a Spark NLP Pipeline by using Spark ML Pipeline natively

In [ ]:
document_assembler = DocumentAssembler() \
    .setInputCol("text")

sentence_detector = SentenceDetector() \
    .setInputCols(["document"]) \
    .setOutputCol("sentence")

tokenizer = Tokenizer() \
    .setInputCols(["sentence"]) \
    .setOutputCol("token")

normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normal")
    
lemmatizer = Lemmatizer() \
    .setInputCols(["normal"]) \
    .setOutputCol("lemma") \
    .setDictionary(
          path = "/tmp/lemma_italian.txt",
          read_as = "TEXT",
          key_delimiter = "\\s+", 
          value_delimiter = "->"
        )
pipeline = Pipeline(stages=[document_assembler, sentence_detector, tokenizer, normalizer, lemmatizer])

Let's see how good our model does when it comes to prediction. We are going to create a DataFrame with Italian text for testing purposes and use `transform()` to predict.

In [6]:
# Let's create a DataFrame with Italian text for testing our Spark NLP Pipeline
dfTest = spark.createDataFrame(["Finchè non avevo la linea ADSL di fastweb potevo entrare nel router e configurare quelle pochissime cose configurabili (es. nome dei device), da ieri che ho avuto la linea niente è più configurabile...", 
    "L'uomo è insoddisfatto del prodotto.", 
    "La coppia contenta si abbraccia sulla spiaggia."], StringType()).toDF("text")

# Of course you can select multiple columns at the same time however, this way we see each annotator without truncating their results
pipeline.fit(dfTest).transform(dfTest).select("token.result").show(truncate=False)
pipeline.fit(dfTest).transform(dfTest).select("normal.result").show(truncate=False)
pipeline.fit(dfTest).transform(dfTest).select("lemma.result").show(truncate=False)

# Print the schema of the Pipeline
pipeline.fit(dfTest).transform(dfTest).printSchema()

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|result                                                                                                                                                                                                                                                 |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[Finchè, non, avevo, la, linea, ADSL, di, fastweb, potevo, entrare, nel, router, e, configurare, quelle, pochissime, cose, configurabili, (, es, ., nome, dei, device, ),, da, ieri, che, ho, avuto, la, linea, niente, è, più, configurabile, ., ., .]|


### Credits 
We would like to thank `DXC.Technology` for sharing their Italian datasets and models with Spark NLP community. The datasets are used to train `Lemmatizer` and `SentimentDetector` Models.